In [ ]:
!pip install gurobipy
from gurobipy import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 38.6 MB/s eta 0:00:00


In [ ]:
teams = ["CAN", "ICE", "SWE", "RUS", "DEN", "JAP", "ANT", "AUS", "CHI", "ARG", "TAH", "FIJ"]
north = ["CAN", "ICE", "SWE", "RUS", "DEN", "JAP"]
south = ["ANT", "AUS", "CHI", "ARG", "TAH", "FIJ"]

weeks = list(range(1, 10))

In [ ]:
#matrix given
matrix = {
    "CAN": {"ICE": 49,  "SWE": 938, "RUS": 1009, "DEN": 710, "JAP": 842, "ANT": 1013, "AUS": 776, "CHI": 1612, "ARG": 1148, "TAH": 1393, "FIJ": 1345},
    "ICE": {"SWE": 981, "RUS": 1050, "DEN": 753, "JAP": 885, "ANT": 1056, "AUS": 819, "CHI": 1655, "ARG": 1192, "TAH": 1927, "FIJ": 1389},
    "SWE": {"RUS": 308, "DEN": 330, "JAP": 97, "ANT": 597, "AUS": 563, "CHI": 1074, "ARG": 897, "TAH": 1416, "FIJ": 1055},
    "RUS": {"DEN": 529, "JAP": 370, "ANT": 833, "AUS": 762, "CHI": 1302, "ARG": 1117, "TAH": 1636, "FIJ": 1275},
    "DEN": {"JAP": 194, "ANT": 385, "AUS": 234, "CHI": 983, "ARG": 667, "TAH": 1273, "FIJ": 841},
    "JAP": {"ANT": 528, "AUS": 427, "CHI": 996, "ARG": 812, "TAH": 1331, "FIJ": 971},
    "ANT": {"AUS": 238, "CHI": 600, "ARG": 306, "TAH": 889, "FIJ": 458},
    "AUS": {"CHI": 837, "ARG": 435, "TAH": 1126, "FIJ": 633},
    "CHI": {"ARG": 542, "TAH": 344, "FIJ": 388},
    "ARG": {"TAH": 736, "FIJ": 199},
    "TAH": {"FIJ": 551},
}

In [ ]:
#distance dictionary
dist = {}
for i in teams:
    for j in teams:
        if i == j:
            dist[(i, j)] = 0
        else:
            d = None
            if i in matrix and j in matrix[i]:
                d = matrix[i][j]
            elif j in matrix and i in matrix[j]:
                d = matrix[j][i]
            if d is None:
                raise ValueError(f"Missing distance between {i} and {j}")
            dist[(i, j)] = d

In [ ]:
print(dist)

{('CAN', 'CAN'): 0, ('CAN', 'ICE'): 49, ('CAN', 'SWE'): 938, ('CAN', 'RUS'): 1009, ('CAN', 'DEN'): 710, ('CAN', 'JAP'): 842, ('CAN', 'ANT'): 1013, ('CAN', 'AUS'): 776, ('CAN', 'CHI'): 1612, ('CAN', 'ARG'): 1148, ('CAN', 'TAH'): 1393, ('CAN', 'FIJ'): 1345, ('ICE', 'CAN'): 49, ('ICE', 'ICE'): 0, ('ICE', 'SWE'): 981, ('ICE', 'RUS'): 1050, ('ICE', 'DEN'): 753, ('ICE', 'JAP'): 885, ('ICE', 'ANT'): 1056, ('ICE', 'AUS'): 819, ('ICE', 'CHI'): 1655, ('ICE', 'ARG'): 1192, ('ICE', 'TAH'): 1927, ('ICE', 'FIJ'): 1389, ('SWE', 'CAN'): 938, ('SWE', 'ICE'): 981, ('SWE', 'SWE'): 0, ('SWE', 'RUS'): 308, ('SWE', 'DEN'): 330, ('SWE', 'JAP'): 97, ('SWE', 'ANT'): 597, ('SWE', 'AUS'): 563, ('SWE', 'CHI'): 1074, ('SWE', 'ARG'): 897, ('SWE', 'TAH'): 1416, ('SWE', 'FIJ'): 1055, ('RUS', 'CAN'): 1009, ('RUS', 'ICE'): 1050, ('RUS', 'SWE'): 308, ('RUS', 'RUS'): 0, ('RUS', 'DEN'): 529, ('RUS', 'JAP'): 370, ('RUS', 'ANT'): 833, ('RUS', 'AUS'): 762, ('RUS', 'CHI'): 1302, ('RUS', 'ARG'): 1117, ('RUS', 'TAH'): 1636, ('R

In [ ]:
for (i, j), d in dist.items():
    print((i, j), d)


In [ ]:
tv_pairs = [("TAH", "FIJ"),
            ("SWE", "DEN"),
            ("CHI", "ARG"),
            ("ANT", "AUS")]

In [ ]:
m = Model("sport_scheduling_WSL")

x_index = [(i, j, w) for i in teams for j in teams if i != j for w in weeks]
x = m.addVars(x_index, vtype=GRB.BINARY, name="x")

r_index = [(i, w) for i in teams for w in weeks[:-1]]
r = m.addVars(r_index, vtype=GRB.BINARY, name="r")

T_max = m.addVar(lb=0.0, name="T_max")
T_min = m.addVar(lb=0.0, name="T_min")

Restricted license - for non-production use only - expires 2027-11-29


In [ ]:
#Constrains

#1. one game per week, atmost
for i in teams:
    for w in weeks:
        m.addConstr(quicksum(x[i, j, w] for j in teams if j != i) +
                    quicksum(x[j, i, w] for j in teams if j != i) <= 1, name=f"one_game_{i}_{w}")

In [ ]:
#2. same division play once with all
#North
for idx_i in range(len(north)):
    for idx_j in range(idx_i + 1, len(north)):
        i, j = north[idx_i], north[idx_j]
        m.addConstr(quicksum(x[i, j, w] + x[j, i, w] for w in weeks) == 1, name=f"division_game_N_{i}_{j}")

#South
for idx_i in range(len(south)):
    for idx_j in range(idx_i + 1, len(south)):
        i, j = south[idx_i], south[idx_j]
        m.addConstr(quicksum(x[i, j, w] + x[j, i, w] for w in weeks) == 1, name=f"division_game_S_{i}_{j}")

In [ ]:
#3 cross division play any three
#atmost one
for i in north:  #change
    for j in south:
        m.addConstr(quicksum(x[i, j, w] + x[j, i, w] for w in weeks) <= 1, name=f"cross_division_pair_{i}_{j}")

#play three random
for i in teams:
    other_div = south if i in north else north
    m.addConstr(quicksum(x[i, j, w] + x[j, i, w] for j in other_div for w in weeks) == 3, name=f"cross_division_count_{i}")

In [ ]:
#4 home 4 games and 4 away games for each team
for i in teams:
    home_games = quicksum(x[i, j, w] for j in teams if j != i for w in weeks)
    away_games = quicksum(x[j, i, w] for j in teams if j != i for w in weeks)
    m.addConstr(home_games == 4, name=f"home_game_count_{i}")
    m.addConstr(away_games == 4, name=f"away_game_count_{i}")


In [ ]:
#5 back to back away not more than once and no start as well as no end with back to back away
for i in teams:
    for w in weeks[:-1]:
        away_w = quicksum(x[j, i, w] for j in teams if j != i)
        away_wplus1 = quicksum(x[j, i, w + 1] for j in teams if j != i)  #changed to w+1
        m.addConstr(r[i, w] >= away_w + away_wplus1 - 1, name=f"back2back_link_{i}_{w}")

    m.addConstr(quicksum(r[i, w] for w in weeks[:-1]) <= 1, name=f"back2back_limit_{i}")

    away_1 = quicksum(x[j, i, 1] for j in teams if j != i)
    away_2 = quicksum(x[j, i, 2] for j in teams if j != i)
    away_8 = quicksum(x[j, i, 8] for j in teams if j != i)
    away_9 = quicksum(x[j, i, 9] for j in teams if j != i)

    m.addConstr(away_1 + away_2 <= 1, name=f"start_not_back2back_{i}")
    m.addConstr(away_8 + away_9 <= 1, name=f"end_not_back2bacak_{i}")

In [ ]:
#6 TV games
for (a, b) in tv_pairs:
    m.addConstr(quicksum(x[a, b, w] + x[b, a, w] for w in weeks) == 1, name=f"tv_once_{a}_{b}")

for w in weeks:
    m.addConstr(quicksum(x[a, b, w] + x[b, a, w] for (a, b) in tv_pairs) <= 1, name=f"tv_one_per_week_{w}")

In [ ]:
#7 travel friendly
travel = {}
for i in teams:
    travel[i] = quicksum(dist[(j, i)] * x[j, i, w] for j in teams if j != i for w in weeks)

for i in teams:
    m.addConstr(T_max >= travel[i], name=f"Tmax_{i}")
    m.addConstr(T_min <= travel[i], name=f"Tmin_{i}")

total_travel = quicksum(travel[i] for i in teams)

In [ ]:
lambda_fair = 0.01
lambda_b2b = 10.0

obj = total_travel + lambda_fair * (T_max - T_min) + lambda_b2b * quicksum(r[i, w] for (i, w) in r_index)

In [ ]:
m.setObjective(obj, GRB.MINIMIZE)

In [ ]:
m.optimize()

if m.Status not in [GRB.OPTIMAL, GRB.INTERRUPTED, GRB.TIME_LIMIT]:
    print(f"Model ended with status {m.Status}")
else:
    print(f"Objective value (weighted): {m.ObjVal:.2f}")
    print("Total travel per team:")
    for i in teams:
        print(f"  {i}: {travel[i].getValue():.1f} miles")

    with open("schedule_WSL.txt", "w") as f:
        for w in weeks:
            for i in teams:
                for j in teams:
                    if i == j:
                        continue
                    if x[i, j, w].X > 0.5:
                        # Line format: HOME AWAY WEEK
                        f.write(f"{i} {j} {w}\n")

    print("Schedule written to schedule_WSL.txt")

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 379 rows, 1286 columns and 12612 nonzeros (Min)
Model fingerprint: 0xe3d2f42d
Model has 1286 linear objective coefficients
Variable types: 2 continuous, 1284 integer (1284 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e-02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]

Presolve removed 28 rows and 24 columns
Presolve time: 0.03s
Presolved: 351 rows, 1262 columns, 11748 nonzeros
Variable types: 0 continuous, 1262 integer (1260 binary)

Root relaxation: objective 3.513017e+04, 1281 iterations, 0.06 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod